In [13]:
import numpy as np
import matplotlib.pyplot as plt
import time
from mpl_toolkits import mplot3d
#%matplotlib notebook
from matplotlib.animation import FuncAnimation

E = 1e4
A = 0.111

nodes = []
bars = []

### Nodes

cas = 1 # 2 ou 3

if cas == 3 : 
    h = 10
    H = 40

    nodes.append([0,0,0])
    nodes.append([1,0,0])
    nodes.append([1,1,0])
    nodes.append([0,1,0])

    nodes.append([0,0,h])
    nodes.append([1,0,h])
    nodes.append([1,1,h])
    nodes.append([0,1,h])

    nodes.append([0.5,0.5,H])

    ### Bars
    #socle 1
    bars.append([0,1])
    bars.append([1,2])
    bars.append([2,3])
    bars.append([3,0])
    #socle 2
    bars.append([4,5])
    bars.append([5,6])
    bars.append([6,7])
    bars.append([7,4])
    #connection 1
    bars.append([0,4])
    bars.append([1,5])
    bars.append([2,6])
    bars.append([3,7])
    #connection 2
    bars.append([4,8])
    bars.append([5,8])
    bars.append([6,8])
    bars.append([7,8])
    #renforcement
    bars.append([0,5])
    bars.append([1,6])
    bars.append([2,7])
    bars.append([3,4])
    
    nodes = np.array(nodes).astype(float)
    bars = np.array(bars)
    
    #Apply force
    P = np.zeros_like(nodes)
    P[8,2] = 5

    # support displacement
    Ur = [0,0,0,0,0,0,0,0,0,0,0,0]

    DOFCON = np.ones_like(nodes).astype(int)
    DOFCON[1,:] = 0
    DOFCON[2,:] = 0
    DOFCON[3,:] = 0
    DOFCON[4,:] = 0
    
elif cas == 2 :
    nodes.append([0,120])
    nodes.append([120,120])
    nodes.append([240,120])
    nodes.append([360,120])
    nodes.append([0,0])
    nodes.append([120,0])
    nodes.append([240,0])
    nodes.append([360,0])
    
    bars.append([0,1])
    bars.append([1,2])
    bars.append([2,3])
    bars.append([4,5])
    bars.append([5,6])
    bars.append([6,7])
    
    bars.append([5,1])
    bars.append([6,2])
    bars.append([7,3])
    
    bars.append([0,5])
    bars.append([4,1])
    bars.append([1,6])
    bars.append([5,2])
    bars.append([2,7])
    bars.append([6,3])
    
    nodes = np.array(nodes).astype(float)
    bars = np.array(bars)
    
    #Apply force
    P = np.zeros_like(nodes)
    P[7,1] = -10

    # support displacement
    Ur = [0,0,0,0]
    
    DOFCON = np.ones_like(nodes).astype(int)
    DOFCON[0,:] = 0
    DOFCON[4,:] = 0
elif cas == 1 :
    a = 1
    ### Nodes
    nodes.append([0,0])
    nodes.append([a,0])
    nodes.append([a/2,np.sqrt(3)/2*a])
    ### Bars
    bars.append([0,1])
    bars.append([1,2])
    bars.append([2,0])
    
    nodes = np.array(nodes).astype(float)
    bars = np.array(bars)
    
    #Apply force
    P = np.zeros_like(nodes)
    P[2,0] = 10

    # support displacement
    Ur = [0,0, 0, 0]
    
    DOFCON = np.ones_like(nodes).astype(int)
    DOFCON[0,:] = 0
    DOFCON[1,:] = 0

print("Nombre d'elements : ",len(bars))

#plt.ion()
def plot3D(nodes,c,lt,lw,lg) : 
    fig = plt.figure(figsize=(8,6))
    #plt.gca(projection='3d')
    ax = fig.add_subplot(111, projection='3d')
    for i in range(len(bars)) :
        xi,xj = nodes[bars[i,0],0],nodes[bars[i,1],0]
        yi,yj = nodes[bars[i,0],1],nodes[bars[i,1],1]
        zi,zj = nodes[bars[i,0],2],nodes[bars[i,1],2]
        line, = ax.plot([xi,xj],[yi,yj],[zi,zj],color = c, lw = lw, linestyle = lt)
        line.set_label(lg)
        #plt.draw()
        #fig.canvas.draw()
        #plt.pause(0.1)
        #ax.cla()
        #fig.canvas.flush_events()
    ax.set_title("Structure")
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_box_aspect([1,1,1])
    geom3D(nodes,ax)
    
def plot2D(nodes,c,lt,lw,lg) : 
    fig = plt.figure(figsize=(8,6))
    ax = fig.add_subplot(111)
    for i in range(len(bars)) :
        xi,xj = nodes[bars[i,0],0],nodes[bars[i,1],0]
        yi,yj = nodes[bars[i,0],1],nodes[bars[i,1],1]
        line, = ax.plot([xi,xj],[yi,yj],color = c, lw = lw, linestyle = lt)
        line.set_label(lg)
        #plt.draw()
        #fig.canvas.draw()
        #plt.pause(0.1)
        #ax.cla()
        #fig.canvas.flush_events()
    ax.set_title("Atom velocity distribution")
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    geom2D(nodes,ax)

def geom3D(NL,ax) : 
    x = [x for x in NL[:,0]]
    y = [y for y in NL[:,1]]
    z = [z for z in NL[:,2]]
    size = 400
    offset = size/4000.
    ax.scatter(x, y, z, c='y', s=size, zorder=1)
    for i, location in enumerate(zip(x,y)):
        #plt.annotate(i+1, (location[0]-offset, location[1]-offset), zorder=10)
        ax.text(x[i],y[i],z[i],str(i+1),size=20,zorder=2,color = "k")

def geom2D(NL,ax) : 
    x = [x for x in NL[:,0]]
    y = [y for y in NL[:,1]]
    size = 400
    offset = size/4000.
    ax.scatter(x, y, c='y', s=size, zorder=1)
    for i, location in enumerate(zip(x,y)):
        #plt.annotate(i+1, (location[0]-offset, location[1]-offset), zorder=10)
        ax.text(x[i],y[i],str(i+1),size=20,zorder=2,color = "k")
        
def truss_analyse(DOF,nodes,bars):
    NN = len(nodes)
    NE = len(bars)
    NDOF = DOF*NN
    d = nodes[bars[:,1],:] - nodes[bars[:,0],:]
    L = np.sqrt((d**2).sum(axis=1))
    angle = d.T/L
    a = np.concatenate((-angle.T,angle.T),axis=1)
    K = np.zeros([NDOF,NDOF])
    for k in range(NE) :
        aux = DOF*bars[k,:]
        index = np.r_[aux[0]:aux[0]+DOF,aux[1]:aux[1]+DOF]
        
        ES = np.dot(a[k][np.newaxis].T*E*A,a[k][np.newaxis]/L[k])
        K[np.ix_(index,index)] = K[np.ix_(index,index)] + ES
        
    print("distance : \n",d,"\n Longeuur \n",L,"\n angle \n",a,"\n Raideur matrice \n",K,"\n")
    
    freeDOF = DOFCON.flatten().nonzero()[0]
    supportDOF = (DOFCON.flatten() == 0).nonzero()[0]
    #print(supportDOF)
    
    Kff = K[np.ix_(freeDOF,freeDOF)]
    Kfr = K[np.ix_(freeDOF,supportDOF)]
    Krf = Kfr.T
    Krr = K[np.ix_(supportDOF,supportDOF)]
    Pf = P.flatten()[freeDOF]
    Uf = np.linalg.solve(Kff,Pf)
    U = DOFCON.astype(float).flatten()
    U[freeDOF] = Uf
    U[supportDOF] = Ur
    U = U.reshape(NN,DOF)
    u = np.concatenate((U[bars[:,0]],U[bars[:,1]]),axis=1)
    N = E*A/L[:]*(a[:]*u[:]).sum(axis=1)
    R = (Krf[:]*Uf).sum(axis=1) + (Krr[:]*Ur).sum(axis=1)
    R = R.reshape(2*(DOF-1),DOF)
    return np.array(N),np.array(R),U

import os
import imageio

filenames = []
    
if cas == 2 :
    N, R, U = truss_analyse(2,nodes,bars)
    print("Axial Forces (positive = tension, negative = compression) :\n", N[np.newaxis].T)
    print("Reaction forces (positive = upward, negative = downward) : \n", R)
    print("Deformation at nodes \n", U)
    plot2D(nodes,'gray','--',1,'undeformed')
    scale = 1
    Dnodes = U*scale + nodes
    plot2D(Dnodes,'red','-',2,'deformed')
    #plt.show()
    #plt.savefig("Customized Surface Plot.png",dpi=300)
elif cas == 3 : 
    plot3D(nodes,'gray','--',1,'undeformed')
    N, R, U = truss_analyse(3,nodes,bars)
    print("Axial Forces (positive = tension, negative = compression) :\n", N[np.newaxis].T)
    print("Reaction forces (positive = upward, negative = downward) : \n", R)
    print("Deformation at nodes \n", U)    
    scale = 1
    Dnodes = U*scale + nodes
    plot3D(Dnodes,'red','-',2,'deformed')
elif cas == 1 :
    plot2D(nodes,'gray','--',1,'undeformed')
    N, R, U = truss_analyse(2,nodes,bars)
    print("Axial Forces (positive = tension, negative = compression) :\n", N[np.newaxis].T)
    print("Reaction forces (positive = upward, negative = downward) : \n", R)
    print("Deformation at nodes \n", U)
    plot2D(nodes,'gray','--',1,'undeformed')
    scale = 1
    Dnodes = U*scale + nodes
    plot2D(Dnodes,'red','-',2,'deformed')
    
import msvcrt

while 0:
    print('Testing..')
    # body of the loop ...
    if msvcrt.kbhit():
        if ord(msvcrt.getch()) == 27:
            break

gif = 0

if gif == 1 :
    path = np.linspace(nodes,U*scale + nodes,100)

    print(path.size)

    for i in range(len(path)):
        plt.cla()
        plot2D(path[i],'red','-',2,'deformed')
        plt.xlim(-5, 400)
        plt.ylim(-30, 150)
        # create file name and append it to a list
        filename = f'{i}.png'
        filenames.append(filename)

        # save frame
        plt.savefig(filename)
        #plt.close()

    #print(path)
        # build gif
    with imageio.get_writer('mygif.gif', mode='I') as writer:
        for filename in filenames:
            image = imageio.imread(filename)
            writer.append_data(image)

    # Remove files
    for filename in set(filenames):
        os.remove(filename)

$MeshFormat
2.2 0 8
$EndMeshFormat
$Nodes
2
7
1 0 0 0
7
2 1 0 0
7
3 2 0 0
7
4 3 0 0
7
5 4 0 0
7
6 0 1 0
7
7 1 1 0
7
8 2 1 0
7
9 3 1 0
8
10 4 1 0
8
11 0 2 0
8
12 1 2 0
8
13 2 2 0
8
14 3 2 0
8
15 4 2 0
8
16 0 3 0
8
17 1 3 0
8
18 2 3 0
8
19 3 3 0
8
20 4 3 0
9
$EndNodes
20
1 0 0 0
2 1 0 0
3 2 0 0
4 3 0 0
5 4 0 0
6 0 1 0
7 1 1 0
8 2 1 0
9 3 1 0
10 4 1 0
11 0 2 0
12 1 2 0
13 2 2 0
14 3 2 0
15 4 2 0
16 0 3 0
17 1 3 0
18 2 3 0
19 3 3 0
20 4 3 0
$EndNodes
$Elements
24
1  2  2  0  1  6  1  2
2  2  2  0  2  7  6  2
3  2  2  0  3  7  11  6
4  2  2  0  4  12  11  7
5  2  2  0  5  16  11  12
6  2  2  0  6  16  12  17
7  2  2  0  7  7  2  3
8  2  2  0  8  8  7  3
9  2  2  0  9  8  12  7
10  2  2  0  10  13  12  8
11  2  2  0  11  17  12  13
12  2  2  0  12  17  13  18
13  2  2  0  13  8  3  4
14  2  2  0  14  9  8  4
15  2  2  0  15  9  13  8
16  2  2  0  16  14  13  9
17  2  2  0  17  18  13  14
18  2  2  0  18  18  14  19
19  2  2  0  19  9  4  5
20  2  2  0  20  10  9  5
21  2  2  0  21  10  14  9

TypeError: 'str' object cannot be interpreted as an integer